# TP1: análisis exploratorio

## Definiciones de funciones generales, imports, y carga de datos

In [ ]:
# magic function para hacer que los graficos de matplotlib se renderizen en el notebook.

import datetime as datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use('bmh') # haciendo los graficos un poco mas bonitos xD
plt.rcParams['figure.figsize'] = (18, 8)
%matplotlib inline

In [ ]:
def show_and_save_plot(title, **kwargs):
    plt.title(title)
    plt.savefig("./imgs/" + title + '.svg', **kwargs)
    plt.show()
    plt.close()

In [ ]:
raw_eventos = pd.read_csv('data/events.csv')

## Preprocesamiento

In [ ]:
## To Do : Usar esto para agregar una nueva feature por marcas.
raw_eventos[pd.notnull(raw_eventos['model']) & ~(raw_eventos.model.str.lower().str.contains('iphone')|\
                                               raw_eventos.model.str.lower().str.contains('samsung') | \
                                               raw_eventos.model.str.lower().str.contains('lg') | \
                                               raw_eventos.model.str.lower().str.contains('motorola')| \
                                               raw_eventos.model.str.lower().str.contains('sony')| \
                                               raw_eventos.model.str.lower().str.contains('asus') | \
                                               raw_eventos.model.str.lower().str.contains('quantum') | \
                                               raw_eventos.model.str.lower().str.contains('ipad') | \
                                               raw_eventos.model.str.lower().str.contains('lenovo') | \
                                               raw_eventos.model.str.lower().str.contains('blackberry') \
                                                )]['model'].unique()

# Funciones de limpieza de datos/creacion de features

In [ ]:
raw_eventos.timestamp = raw_eventos.timestamp.map(pd.Timestamp)
raw_eventos.time = raw_eventos.timestamp.map(lambda it: it.hour)
def megapixel_count(resolution):
    if type(resolution) != str: return np.NaN
    x_pixels, y_pixels = map(int, resolution.split("x"))
    return x_pixels*y_pixels/1000000
raw_eventos.screen_megapixels = raw_eventos.screen_resolution.map(megapixel_count)

# Introduccion a los datos

In [ ]:
raw_eventos.event.value_counts().plot(kind='barh',logx=True)
show_and_save_plot('Tipos de eventos')

In [ ]:
# Voy a intentar ver que proporcion de cada columna es nula
# Esto debería tener relación con que tipos de eventos tienen que atributos.
# Probablemente esta informacion salga de una db con un mapeo single_table
def myIsNa(serie):
    return serie.isna().sum() / len(serie.isna())
serie = pd.Series([1, 2, 3, np.NaN])
serie.isna().count()
grouped = raw_eventos.groupby('event').agg(myIsNa)
sns.heatmap(grouped)
# TODO: sacar la barra de color, confunde.
show_and_save_plot('Proporcion de columnas nulas por evento', pad_inches=0.3, bbox_inches='tight')
del grouped

# Exploracion de los distintos tipos de eventos por separado

## Eventos de busqueda

In [ ]:
# Podemos ver que la columna 'sku', se corresponde con los tipos de evento relacionados a un solo dispositivo
# Por otro lado, la columna 'skus', tiene un string que se corresponde con un listado de dispositivos.
busquedas = raw_eventos[raw_eventos.event == 'searched products'][['skus','timestamp','person','search_term']]
# verifico que para el mismo usuario y search_term hay resultados distintos, para validar lo anterior
busquedas[(busquedas.search_term.isna()) & (busquedas.person == '00aaa92e')]
del busquedas

## Eventos de visited_site

In [ ]:
raw_eventos.channel.value_counts().plot(kind='barh',logx=True)
show_and_save_plot('Tipos de canales')

In [ ]:
raw_eventos.device_type.value_counts().plot(kind='barh',logx=True)
show_and_save_plot('Tipos de dispositivos')

In [ ]:
# Vemos que las resoluciones de pantalla son en general bajas, lo que da la idea
# de que la aplicación es usada por personas de bajo poder adquisitivo.
raw_eventos.screen_resolution.value_counts()[0:15].plot(kind='barh',logx=True)
show_and_save_plot('tipos de resoluciones')

# Exploracion de todos los eventos en conjunto

In [ ]:
# Nos damos una idea de qué resoluciones de pantalla tienen los usuarios
raw_eventos.screen_megapixels.hist(bins=[x*0.3 for x in range(10)])
plt.xlabel('Megapixeles')
show_and_save_plot('Distribución de cantidad de pixeles de las pantallas')

In [ ]:
raw_eventos.timestamp.hist()
show_and_save_plot("distribución de eventos en el tiempo")

In [ ]:
# Cuantos 'skus' distintos hay
print( "hay {} dispositivos y {} dispositivos distintos".format(len(raw_eventos.skus.dropna().values), len(raw_eventos.skus.unique())))

In [ ]:
# Hora del dia en la que se producen los eventos:
raw_eventos.time.hist(bins=24)
show_and_save_plot('Distribucion de eventos en las horas del dia')

## Relación entre eventos de un mismo usuario
Primero voy a ver cuantos eventos hay por cada usuario ¿es usual que haya cientos por cada uno? ¿Solo un par?

In [ ]:
raw_eventos.person.value_counts().hist(log=True, bins=[i*10 for i in range(100)])
show_and_save_plot('cantidad de eventos por persona', xlabel='cantidad de usuarios', ylabel='Cantidad de eventos')

In [ ]:
# TODO: plotear qué tanto tiempo pasan los usuarios en el sitio
persons = raw_eventos.groupby('person').agg({'timestamp':['min','max','count']})
persons['time_spent'] = persons['timestamp','max'] - persons['timestamp','min']
persons.time_spent = persons.time_spent.map(lambda it: it.total_seconds())
persons.time_spent.plot(kind='box')

## Features de cada modelo

In [ ]:
def get_models_data(raw_events):
    def event_count_by_type(event_type, serie):
        return int(serie[serie == event_type].count())
    def viewed_product(it):
        return event_count_by_type('viewed product',it)
    def checkout(it):
        return event_count_by_type('checkout',it)
    def conversion(it):
        return event_count_by_type('conversion',it)
    def lead(it):
        return event_count_by_type('lead',it)
    event_functions = [ viewed_product, checkout, conversion, lead]
    models = raw_events\
        .groupby('model')\
        .agg({'timestamp':['count','min','max'],'event':event_functions})
    models['cantidad_eventos'] = models['timestamp','count']
    models = models.drop(labels=[('timestamp','count')],axis=1)
    models = models[models.cantidad_eventos > 100]
    for i in ['checkout', 'conversion','lead','viewed_product']:
        models['event',i] = models['event',i]/models['cantidad_eventos']
    return models

In [ ]:
models = get_models_data(raw_eventos).sort_values('cantidad_eventos', ascending=False)
models.event.head(8).plot(kind='barh',logx=True, stacked=False)
show_and_save_plot('Distribucion de eventos en los modelos mas populares', xlabel='cantidad  de eventos')

Modelos de telefono mas vendidos en proporcion a los eventos totales

Vale la pena aclarar que son modelos distintos a los mas populares

In [ ]:
models = models.sort_values([('event','conversion')], ascending=False)
models.event.head(8).plot(kind='barh',logx=True, stacked=False)
show_and_save_plot('Distribucion de eventos en los modelos con mejor conversion rate', xlabel='cantidad  de eventos')

In [ ]:
models = models.sort_values([('event','lead')], ascending=False)
models.event.head(8).plot(kind='barh',logx=True, stacked=False)
show_and_save_plot('Distribucion de eventos en los modelos mas reservados', xlabel='cantidad  de eventos')

## Evaluo cuales son los colores mas vendidos.

In [ ]:
colors_count = raw_eventos[raw_eventos.event == 'conversion'].color.dropna().value_counts()

In [ ]:
colors_count.plot(kind = 'barh')